<a href="https://colab.research.google.com/github/snghynn/ESAA-YB/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
!pip install catboost==1.1.1
from collections import defaultdict

In [ ]:
import numpy as np
import pandas as pd

## 1. 데이터 불러오기 및 탐색

In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [ ]:
train.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN


- 1,000,000개의 데이터
- ID : 샘플 고유 id, 항공편 운항 관련 정보
- Month: 해당 항공편의 출발 월
- Day_of_Month: Month에 해당하는 월의 날짜
- Estimated_Departure_Time: 전산 시스템을 바탕으로 측정된 비행기의 출발 시간 (현지 시각, HH:MM 형식)
- Estimated_Arrival_Time: 전산 시스템을 바탕으로 측정된 비행기의 도착 시간 (현지 시각, HH:MM 형식)
- Cancelled: 해당 항공편의 취소 여부 (0: 취소되지 않음, 1: 취소됨)
- Diverted: 해당 항공편의 경유 여부 (0: 취소되지 않음, 1: 취소됨)
- Origin_Airport: 해당 항공편 출발 공항의 고유 코드 (IATA 공항 코드)
- Origin_Airport_ID: 해당 항공편 출발 공항의 고유 ID (US DOT ID)
- Origin_State: 해당 항공편 출발 공항이 위치한 주의 이름
- Destination_Airport: 해당 항공편 도착 공항의 고유 코드 (IATA 공항 코드)
- Destination_Airport_ID: 해당 항공편 도착 공항의 고유 ID (US DOT ID)
- Destination_State: 해당 항공편 도착 공항이 위치한 주의 이름
- Distance: 출발 공항과 도착 공항 사이의 거리 (mile 단위)
- Airline: 해당 항공편을 운항하는 항공사
- Carrier_Code(IATA): 해당 항공편을 운항하는 항공사의 고유 코드
 (IATA 공항 코드, 단 다른 항공사가 같은 코드를 보유할 수도 있음)
- Carrier_ID(DOT): 해당 항공편을 운항하는 항공사의 고유 ID (US DOT ID)
- Tail_Number: 해당 항공편을 운항하는 항공기의 고유 등록번호
- Delay: 항공편 지연 여부 (Not_Delayed, Delayed)
    - 예측해야 하는 타깃
    - 다수의 데이터에 레이블이 존재하지 않음
     -준지도학습을 통해 레이블링 가능

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 19 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport            1000000 non-null  object 
 8   Origin_Airport_ID         1000000 non-null  int64  
 9   Origin_State              890985 non-null   object 
 10  Destination_Airport       1000000 non-null  object 
 11  Destination_Airport_ID    1000000 non-null  int64  
 12  Destination_State         890921 non-null   object 
 13  Distance                  10

In [ ]:
test.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number
0,TEST_000000,12,16,1156.0,NaN,0,0,IAH,12266,Texas,SAT,14683,Texas,191.0,United Air Lines Inc.,UA,NaN,N79402
1,TEST_000001,9,12,1500.0,1715.0,0,0,EWR,11618,New Jersey,ATL,10397,NaN,746.0,Delta Air Lines Inc.,DL,19790.0,N3765
2,TEST_000002,3,6,1600.0,1915.0,0,0,ORD,13930,Illinois,LGA,12953,New York,733.0,United Air Lines Inc.,UA,19977.0,N413UA
3,TEST_000003,5,18,1920.0,2045.0,0,0,OAK,13796,California,LAX,12892,California,337.0,Southwest Airlines Co.,WN,19393.0,N905WN
4,TEST_000004,7,7,1915.0,2152.0,0,0,FLL,11697,Florida,LAX,12892,California,2343.0,JetBlue Airways,B6,20409.0,N945JT


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 18 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  891016 non-null   float64
 4   Estimated_Arrival_Time    890952 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport            1000000 non-null  object 
 8   Origin_Airport_ID         1000000 non-null  int64  
 9   Origin_State              893495 non-null   object 
 10  Destination_Airport       1000000 non-null  object 
 11  Destination_Airport_ID    1000000 non-null  int64  
 12  Destination_State         893477 non-null   object 
 13  Distance                  10

## 2. 데이터 전처리

> train, test data 둘 다에서 다수의 결측값이 존재한다. 어떻게 처리할까?
>
> : 비행기 연착과 관련된 데이터의 특성을 고려하자

### 1) Carrier_ID(DOT)
- Carrier_ID(DOT)은 해당 항공사(Airline) 별로 하나 할당. 둘 중 하나만 있어도 분류하는데 지장이 없음, 각각 891080개의 non-null 값을 보면, 두 개 모두 결측치가 아닌 이상, 데이터 복구가 가능
- 복구 가능한 데이터의 개수 (Airline이 존재하면서, Carrier_ID(DOT)이 존재 안하는 row의 개수)를 확인한 후, Airline과 Carrier_ID(DOT)을 이어주는 dictionary 를 만들어서 가능한 Carrier_ID(DOT)를 복구

In [ ]:
# Airline 을 통해 Carrier_ID(DOT) 복구 가능한 데이터의 개수 파악 및 복구
cond1 = train['Carrier_ID(DOT)'].isnull()
cond2 = train['Airline'].isnull()
print("Carrier_ID(DOT) 복구 가능한 데이터의 개수 :", len(train.loc[cond1 & cond2, :]))

Carrier_ID(DOT) 복구 가능한 데이터의 개수 : 11883


In [ ]:
# airline to carrier id, dictinary 만들기
# 모두 데이터가 존재하는 열에서 Dict[Airline] = carrier_ID(DOT) 가 되도록 dictionary 생성
airline_to_cid = {}
for _, row in train[(train['Carrier_ID(DOT)'].isnull() & train['Airline'].isnull())].iterrows():
    airline_to_cid[row['Airline']] = row['Carrier_ID(DOT)']

In [ ]:
# Carrier_ID(DOT) 복구
def to_cid(x):
    return airline_to_cid[x]

cond1 = train['Carrier_ID(DOT)'].isnull()
cond2 = train['Airline'].isnull()
train.loc[cond1&cond2, 'Carrier_ID(DOT)'] = train.loc[cond1&cond2, 'Airline'].apply(lambda x: to_cid(x))

In [ ]:
# 복구 안 된 Carrier_ID는 학습에 방해되므로 제거
train = train.dropna(subset=['Carrier_ID(DOT)'], how='any', axis=0)

- test data의 경우 복구 안 된 row를 빼 버리면 안되기에, Airline, Carrier_ID(DOT) 둘 다 없는 row는 Carrier_ID(DOT)에 최빈 값의 Carrier_ID(DOT) 대체
- Airline 으로부터 복구 가능한 Carrier_ID(DOT)은 Training과 마찬가지로 복구함

In [ ]:
# (Test Data Only)
# Airline, Carrier_Code 둘 다 없으면 최빈 값으로 대체
NaN_col = ['Carrier_ID(DOT)']
cond1 = test['Airline'].isnull()
cond2 = test['Carrier_ID(DOT)'].isnull()

for col in NaN_col:
    mode = test[col].mode()[0]
    test.loc[cond1&cond2, col] = mode

# 나머진 Airline에서 대체
cond1 = test['Carrier_ID(DOT)'].isnull()
cond2 = test['Airline'].isnull()
test.loc[cond1&cond2, 'Carrier_ID(DOT)'] = test.loc[cond1&cond2, 'Airline'].apply(lambda x: to_cid(x))

### 2) 제거할 Columns 제거
- Cancelled, Diverted 는 모두 0이므로 의미없는 값이므로 제거
- Origin_Airport, Destination_Airport 는 Origin_Airport_ID와 Destination_Airport_ID와 중복 -> 제거
- Carrier_Code(IATA)는 Airline 별로 N:1로 할당 된 값이므로, 큰 의미가 없는 것으로 판단하여 제거
- State 정보 보다는 Origin_Airport_ID가 중요하다고 판단해 (미국 주는 너무 커서 의미가 없다고 생각했음) 제거

In [ ]:
col_drop = ['Cancelled', 'Diverted', 'Origin_Airport', 'Destination_Airport', 'Carrier_Code(IATA)', 'Airline', 'Origin_State', 'Destination_State']
train = train.drop(col_drop, axis=1)
test = test.drop(col_drop, axis=1)

### 3) Estimated Departure Time (EDT), Estimated Arrival Time (EAT)

#### (1) 복구

: 결측치가 없는 Origin_Airport_ID → Destination_Airport_ID 의 정보를 이용

In [ ]:
# Arrival Time & Departure Time 둘 다 60분 * 24 = 1440 로 바꾸기
def to_minutes(x):
    x = int(x)
    x = str(x)
    if len(x) > 2:
        hours, mins = int(x[:-2]), int(x[-2:])
    else:
        hours, mins = 0, int(x[-2:])
    return hours*60+mins

estimated_times = ['Estimated_Departure_Time', 'Estimated_Arrival_Time']

for ET in estimated_times:
    cond = ~train[ET].isnull()
    train.loc[cond, ET] = train.loc[cond, ET].apply(lambda x: to_minutes(x))
    cond2 = ~test[ET].isnull()
    test.loc[cond2, ET] = test.loc[cond2, ET].apply(lambda x: to_minutes(x))

In [ ]:
# EAT, EDT 모두 없는 값은 Training에 도움이 안 되므로 빼기
train = train.dropna(subset=['Estimated_Arrival_Time', 'Estimated_Departure_Time'], how ='all', axis=0)

In [ ]:
# (OAID, DAID)를 키로 갖고, 평균 비행시간을 값으로 갖는 dictionary 만들기
time_flying = defaultdict(int)
time_number = defaultdict(int)

cond_arr2 = ~train['Estimated_Arrival_Time'].isnull()
cond_dep2 = ~train['Estimated_Departure_Time'].isnull()

for _, row in train.loc[cond_arr2 & cond_dep2, :].iterrows():
    OAID, DAID = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    time_flying[(OAID,DAID)] += (row['Estimated_Arrival_Time'] - row['Estimated_Departure_Time'])%1440 # 하루 최대는 1440분
    time_number[(OAID,DAID)] += 1


for key in time_flying.keys():
    time_flying[key] /= time_number[key]

In [ ]:
# EAT, EDT 모두 없는 값은 Training에 도움이 안 되므로 빼기
train = train.dropna(subset=['Estimated_Arrival_Time', 'Estimated_Departure_Time'], how ='all', axis=0)

In [ ]:
# (OAID, DAID)를 키로 갖고, 평균 비행시간을 값으로 갖는 dictionary 만들기
time_flying = defaultdict(int)
time_number = defaultdict(int)

cond_arr2 = ~train['Estimated_Arrival_Time'].isnull()
cond_dep2 = ~train['Estimated_Departure_Time'].isnull()

for _, row in train.loc[cond_arr2 & cond_dep2, :].iterrows():
    OAID, DAID = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    time_flying[(OAID,DAID)] += (row['Estimated_Arrival_Time'] - row['Estimated_Departure_Time'])%1440 # 하루 최대는 1440분
    time_number[(OAID,DAID)] += 1


for key in time_flying.keys():
    time_flying[key] /= time_number[key]

In [ ]:
# Dictionary를 이용해 EAT, EDT 복구
for index, row in train.loc[train['Estimated_Departure_Time'].isnull(),].iterrows():
    OAID, DAID = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    train.loc[index,'Estimated_Departure_Time'] = \
        (train.loc[index]['Estimated_Arrival_Time'] - time_flying[(OAID, DAID)])%1440

for index, row in train.loc[train['Estimated_Arrival_Time'].isnull(),].iterrows():
    OAID, DAID = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    train.loc[index,'Estimated_Arrival_Time'] = \
        (train.loc[index]['Estimated_Departure_Time'] + time_flying[(OAID, DAID)])%1440

In [ ]:
# Test Data 처리: EAT, EDT 둘 다 없는 row는 각각의 최빈값을 채워주도록 하고 나머지는 복구
# (Test Data Only)
# 둘 다 없으면 최빈값으로 대체
cond_1 = test['Estimated_Departure_Time'].isnull()
cond_2 = test['Estimated_Arrival_Time'].isnull()

mode = test['Estimated_Departure_Time'].mode()[0]
mode2 = test['Estimated_Arrival_Time'].mode()[0]
test.loc[cond_1&cond_2, ['Estimated_Departure_Time', 'Estimated_Arrival_Time']] = mode, mode2


# Departure만 없을 경우
for index, row in test.loc[test['Estimated_Departure_Time'].isnull(),].iterrows():
    OAID, DAID = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    test.loc[index,'Estimated_Departure_Time'] = \
        (test.loc[index]['Estimated_Arrival_Time'] - time_flying[(OAID, DAID)])%1440


# Arrival만 없을 경우
for index, row in test.loc[test['Estimated_Arrival_Time'].isnull(),].iterrows():
    OAID, DAID = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    test.loc[index,'Estimated_Arrival_Time'] = \
        (test.loc[index]['Estimated_Departure_Time'] + time_flying[(OAID, DAID)])%1440


# 모두 int로 바꿈
estimated_times = ['Estimated_Departure_Time', 'Estimated_Arrival_Time']
train = train.astype({'Estimated_Departure_Time':int, 'Estimated_Arrival_Time':int})
test = test.astype({'Estimated_Departure_Time':int, 'Estimated_Arrival_Time':int})
for ET in estimated_times:
    train.loc[train[ET] == 1440, ET] = 0
    test.loc[test[ET] == 1440, ET] = 0

In [ ]:
train.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay
1,TRAIN_000001,8,15,460,624,13930,14869,1250.0,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,970,1085,11057,12953,544.0,19805.0,N103US,NaN
4,TRAIN_000004,1,11,540,619,14771,10157,250.0,20304.0,N161SY,NaN
5,TRAIN_000005,4,13,945,1026,11618,11278,199.0,20452.0,N657RW,Not_Delayed
7,TRAIN_000007,4,20,1095,1195,13930,13198,403.0,20304.0,N110SY,NaN


#### (2) group
: 특정 시간이 비행기 연착에 영향을 미칠 것으로 판단. EDT, EAT경우 30분 단위로 48개의 bin에 담아서 object화 진행

-> ordinal한 관계는 전혀 없으면서 비슷한 시간대(30분 단위)는 같은 특성을 갖도록.

In [ ]:
estimate_times = ['Estimated_Departure_Time', 'Estimated_Arrival_Time']
names = {'Estimated_Departure_Time':'EDT', 'Estimated_Arrival_Time':'EAT'}
for ET in estimated_times:
    for i in range(48):
        train.loc[train[ET].between(i*30, (i+1)*30, 'left'), names[ET]] = i
        test.loc[test[ET].between(i*30, (i+1)*30, 'left'), names[ET]] = i

In [ ]:
train['EDT'] = train['EDT'].astype(int)
train['EAT'] = train['EAT'].astype(int)
test['EDT'] = test['EDT'].astype(int)
test['EAT'] = test['EAT'].astype(int)

In [ ]:
train = train.drop(['Estimated_Departure_Time', 'Estimated_Arrival_Time'], axis=1)
test = test.drop(['Estimated_Departure_Time', 'Estimated_Arrival_Time'], axis=1)